In [2]:
import sys, os
import random
import numpy as np
from glob import glob
import pandas as pd
import kaldiio
import torch


random.seed(1)

print(torch.version)
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(device)

in_set = ['ENG', 'GER', 'ICE', 'FRE', 'SPA', 'ARA', 'RUS', 'BEN', 'KAS', 'GRE', 'CAT', 'KOR', 'TUR', 'TAM', 'TEL', 'CHI', 'TIB', 'JAV', 'EWE', 'HAU', 'LIN', 'YOR', 'HUN', 'HAW', 'MAO', 'ITA', 'URD', 'SWE', 'PUS', 'GEO', 'HIN', 'THA']
out_of_set = ['DUT', 'HEB', 'UKR', 'BUL', 'PER', 'ALB', 'UIG', 'MAL', 'BUR', 'IBA', 'ASA', 'AKU', 'ARM', 'HRV', 'FIN', 'JPN', 'NOR', 'NEP', 'RUM']

langs = in_set + out_of_set

num_in_set = 32

in_set = langs[:num_in_set]
out_of_set = langs[num_in_set:]

root_dir = "/home/sp4013/kaldi-trunk/egs/lid/s1/db/cu-multilang-dataset/"


assert(len(in_set) + len(out_of_set) == 51)
assert(len(set(in_set).intersection(set(out_of_set))) == 0)
for lang in os.listdir(root_dir):
    assert(lang in in_set or lang in out_of_set)

# print("\n-----Getting Train/Test Data-----\n")

# train, test = [], []
# max_sample_length = 300
# num_chunks_per_file = 120
# num_files = 30

# for i,lang in enumerate(in_set + out_of_set, 0):
#     print(lang, "(In-set)" if lang in in_set else "(Out-of-set)")
    
#     chunks = []
#     for f_idx in range(1, num_files + 1):
        
#         filepath = root_dir + lang + '/data/raw_mfcc_pitch_' + lang + '.' + str(f_idx) + '.ark'
#         file_chunks = []
        
#         for key, numpy_array in kaldiio.load_ark(filepath):
#             curr_len = len(numpy_array)

# #             chunks +=  np.split(numpy_array, np.arange(max_sample_length, curr_len, max_sample_length))
#             if curr_len >= max_sample_length:
#                 file_chunks +=  np.split(numpy_array, np.arange(max_sample_length, curr_len, max_sample_length))[:-1]
# #             else:
# #                 padded_chunk = np.pad(numpy_array, ((max_sample_length - curr_len, 0), (0, 0)), "constant")
# #                 file_chunks += [padded_chunk]
                
# #             if len(file_chunks) >= num_chunks_per_file:
#                 chunks += file_chunks
# #                 break
    
#     random.shuffle(chunks)

#     # Put chunks in train or test depending on in_set or out_of_set
#     switch_point = 0.80 if lang in in_set else 0.0
#     for j in range(len(chunks)):
#         chunk = chunks[j]
#         inputs = torch.from_numpy(np.expand_dims(chunk, axis=0))
#         inputs.to(device)
#         labels = torch.from_numpy(np.array([i if lang in in_set else -1]))
#         labels.to(device)

#         if j+1 <= switch_point * len(chunks):
#             train.append((inputs,labels))
#         else:
#             test.append((inputs,labels))

# print()
# print("Num chunks in training:", len(train))
# print("Num chunks in testing:", len(test))

# print("\n-----Finished Data Splitting-----\n")

<module 'torch.version' from '/home/sp4013/anaconda3/lib/python3.9/site-packages/torch/version.py'>
cuda:0


In [4]:
from collections import defaultdict


data = []
for i,lang in enumerate(in_set + out_of_set, 0):
    print(lang, "(In-set)" if lang in in_set else "(Out-of-set)")
    
    for f_idx in range(1, 30 + 1):
        
        filepath = root_dir + lang + '/data/raw_mfcc_pitch_' + lang + '.' + str(f_idx) + '.ark'

        for key, numpy_array in kaldiio.load_ark(filepath):
            inputs = torch.from_numpy(np.expand_dims(numpy_array, axis=0))
            inputs.to(device)
            labels = torch.from_numpy(np.array([i if lang in in_set else -1]))
            labels.to(device)
            data.append((inputs, labels))


print("combining all mfccs by label")
random.shuffle(data)
data_concatenated = dict()
for iter,i in enumerate(data): 
    label = i[1].numpy()[0]
    mfcc = np.squeeze(i[0].numpy(), axis=0)
    if label in data_concatenated:
        data_concatenated[label].append(mfcc)
    else:
        data_concatenated[label] = [mfcc]

for i in data_concatenated:
    data_concatenated[i] = np.vstack(data_concatenated[i])

del data

def chunkify_tensor(tensor, size=400):
    return torch.split(tensor, size, dim=1)[:-1] # except last one bc that isn't the right size

'''
Train (tdnn):       95% of in-set
Train (lda/plda):   80% of out-of-set
Test:               5% of in-set + 20% out-of-set
'''

print("chunking and splitting into 3 groups")
train1, train2, test = [], [], []
for i in data_concatenated:
    label = torch.from_numpy(np.array([i]))
    mfcc = torch.from_numpy(np.expand_dims(data_concatenated[i], axis=0))
    chunks = chunkify_tensor(mfcc)
    if i >= 0:
        cutoff = int(len(chunks) * 0.95)
        for chunk in chunks[:cutoff]:
            train1.append((chunk.to(device), label.to(device)))
        for chunk in chunks[cutoff:]:
            test.append((chunk.to(device), label.to(device)))
    else:
        cutoff = int(len(chunks) * 0.8)
        for chunk in chunks[:cutoff]:
            train2.append((chunk.to(device), label.to(device)))
        for chunk in chunks[cutoff:]:
            test.append((chunk.to(device), label.to(device)))

del data_concatenated

print()
print("train1:", len(train1))
print("train2:", len(train2))
print("test:", len(test))
print("\ttest (in-set):", len([i for i in test if i[1].to('cpu').numpy()[0] >= 0]))
print("\ttest (oos):", len([i for i in test if i[1].to('cpu').numpy()[0] < 0]))
print()
print("labels in test:", set([i[1].to('cpu').numpy()[0] for i in test]))
print(test[0][0].size())
asdfasdfasdf = defaultdict(int)
for i in test:
    asdfasdf = i[1].to('cpu').numpy()[0]
    asdfasdfasdf[asdfasdf] += 1
print(sorted(asdfasdfasdf.items()))
print()

ENG (In-set)
GER (In-set)
ICE (In-set)
FRE (In-set)
SPA (In-set)
ARA (In-set)
RUS (In-set)
BEN (In-set)
KAS (In-set)
GRE (In-set)
CAT (In-set)
KOR (In-set)
TUR (In-set)
TAM (In-set)
TEL (In-set)
CHI (In-set)
TIB (In-set)
JAV (In-set)
EWE (In-set)
HAU (In-set)
LIN (In-set)
YOR (In-set)
HUN (In-set)
HAW (In-set)
MAO (In-set)
ITA (In-set)
URD (In-set)
SWE (In-set)
PUS (In-set)
GEO (In-set)
HIN (In-set)
THA (In-set)
DUT (Out-of-set)
HEB (Out-of-set)
UKR (Out-of-set)
BUL (Out-of-set)
PER (Out-of-set)
ALB (Out-of-set)
UIG (Out-of-set)
MAL (Out-of-set)
BUR (Out-of-set)
IBA (Out-of-set)
ASA (Out-of-set)
AKU (Out-of-set)
ARM (Out-of-set)
HRV (Out-of-set)
FIN (Out-of-set)
JPN (Out-of-set)
NOR (Out-of-set)
NEP (Out-of-set)
RUM (Out-of-set)
combining all mfccs by label
chunking and splitting into 3 groups

train1: 246728
train2: 86336
test: 34586
	test (in-set): 13001
	test (oos): 21585

labels in test: {0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 

In [7]:
chunk_count = {}

for x, y in test:
    i = y.item()
    if i not in chunk_count:
        chunk_count[i] = 1
    else:
        chunk_count[i] += 1

print(chunk_count)

{19: 449, -1: 21585, 6: 449, 3: 449, 15: 448, 5: 450, 13: 318, 4: 450, 0: 213, 30: 226, 7: 448, 2: 449, 24: 449, 17: 449, 23: 449, 31: 449, 28: 450, 1: 449, 16: 450, 18: 450, 11: 449, 9: 177, 29: 450, 25: 450, 20: 450, 14: 256, 22: 450, 12: 450, 26: 450, 27: 450, 21: 450, 10: 423, 8: 152}


In [3]:
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
import torch.optim as optim
from tdnn import TDNN
from pytorch_model_summary import summary


class Net(nn.Module):
    def __init__(self, in_size, num_classes):
        super().__init__()
        
        self.layer1 = TDNN(input_dim=in_size, output_dim=256, context_size=5)
        self.layer2 = TDNN(input_dim=256, output_dim=256, context_size=3, dilation=2)
        self.layer3 = TDNN(input_dim=256, output_dim=256, context_size=3, dilation=3)
        self.layer4 = TDNN(input_dim=256, output_dim=256, context_size=1)
        self.layer5 = TDNN(input_dim=256, output_dim=256, context_size=1)
        self.final_layer = TDNN(input_dim=256, output_dim=num_classes, context_size=1)
        
    def forward(self, x):
        forward_pass = nn.Sequential(
            self.layer1,
            nn.ReLU(),
            self.layer2,
            nn.ReLU(),
            self.layer3,
            nn.ReLU(),
            self.layer4,
            nn.ReLU(),
            self.layer5,
            nn.ReLU(),
            self.final_layer)
        
        return forward_pass(x)
    
print(summary(Net(16, len(in_set)), torch.zeros((1, 400, 16)), show_input=False))

-----------------------------------------------------------------------
      Layer (type)        Output Shape         Param #     Tr. Param #
            TDNN-1       [1, 396, 256]          21,248          21,248
            TDNN-2       [1, 392, 256]         197,376         197,376
            TDNN-3       [1, 386, 256]         197,376         197,376
            TDNN-4       [1, 386, 256]          66,304          66,304
            TDNN-5       [1, 386, 256]          66,304          66,304
            TDNN-6        [1, 386, 32]           8,288           8,288
Total params: 556,896
Trainable params: 556,896
Non-trainable params: 0
-----------------------------------------------------------------------


In [9]:
from pickle import Pickler, Unpickler

infile = open("saved_models/tdnn-256-4s-25epochs.pickle", "rb")
new_net = Unpickler(infile).load()
infile.close()

# new_net = Net(16, 32)
# new_net.load_state_dict(torch.load("saved_models/tdnn-256-4s.pth"))
# new_net.to(device)

# out_len = len(test)
# count = 0
# for x, y in val_set:
#     test.append((x.unsqueeze(0), y))
#     count += 1
#     if count == out_len:
#         break
    
print(len(test))

for thresh in [0.0, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]:
    correct = 0
    for x, y in test:
      # Send to GPU if available
      if device is not None:
        x = x.to(device)
        y = y.to(device)

      y_pred = new_net(x)
      y_pred = torch.mean(y_pred, 1)
#       top_pred = y_pred.argmax(1, keepdim=True)
      y_pred = F.softmax(y_pred, dim=1)
#       print(y_pred)
    
      conf = y_pred.amax(1, keepdim=True).item()
      if conf > thresh:
          top_pred = y_pred.argmax(1, keepdim=True)
      else:
          top_pred = -1

      if top_pred == y:
        correct += 1
    
    print(thresh, correct / len(test))
    

34586
0.0 0.3478575146012838
0.4 0.7059214711154802
0.5 0.7727404151969005
0.6 0.8186260336552362
0.7 0.8499681952235009
0.8 0.8705834730815937
0.9 0.8764818134505291


In [3]:
def create_batches(data, batch_size=64):
    len_data = len(data)
    num_batches = (len_data // batch_size) + 1

    for i in range(num_batches):
        start_idx = i * batch_size
        end_idx = min(start_idx + batch_size, len_data - 1)

        yield data[start_idx: end_idx]

In [4]:
def calculate_accuracy(y_pred, y):
    top_pred = y_pred.argmax(1, keepdim=True)
    correct = top_pred.eq(y.view_as(top_pred)).sum()
    acc = correct.float() / y.shape[0]
    return acc


def train_model(model, loader, optimizer, criterion, device=None):
    train_loss = 0.0
    train_acc = 0.0
    num_batches = 0

    model.train()

    for batch in loader:
        num_batches += 1
        len_batch = len(batch)
        optimizer.zero_grad()
        loss = 0.0
        acc = 0.0
        for x, y in batch:
          # Send to GPU if available
          if device is not None:
            x = x.to(device)
            y = y.to(device)

          y_pred = model(x)
          y_pred = torch.mean(y_pred, 1)
          loss += criterion(y_pred, y)
          acc += calculate_accuracy(y_pred, y)
        
        loss /= len_batch
        acc /= len_batch
        loss.backward()
        optimizer.step()

        train_loss += loss.item()
        train_acc += acc.item()

    return train_loss / num_batches, train_acc / num_batches


def evaluate_model(model, loader, criterion, device=None):
    val_loss = 0.0
    val_acc = 0.0
    num_batches = 0

#     model.eval()
    with torch.no_grad():
        for batch in loader:
            num_batches += 1
            len_batch = len(batch)
            loss = 0.0
            acc = 0.0

            for x, y in batch:
              # Send to GPU if available
              if device is not None:
                x = x.to(device)
                y = y.to(device)

              y_pred = model(x)
              y_pred = torch.mean(y_pred, 1)
              loss += criterion(y_pred, y)
              acc += calculate_accuracy(y_pred, y)

            loss /= len_batch
            acc /= len_batch

            val_loss += loss.item()
            val_acc += acc.item()

    return val_loss / num_batches, val_acc / num_batches

In [5]:
# Initialize the TDNN, loss, and optimizer
# print(len(in_set))
net = Net(16, len(in_set))
net.to(device)
criterion = nn.CrossEntropyLoss() # a common loss function for multi-class classification problems
optimizer = optim.Adam(net.parameters(), lr=0.001) # a common optimizer for multi-class classification problems


# Train the TDNN network
do_training = True # whether to train or to load a saved model
SAVE_PATH = 'saved_models/tdnn.pth'
LOAD_PATH = 'saved_models/tdnn.pth'

train_losses = []
train_accuracies = []
val_losses = []
val_accuracies = []
best_loss = float('inf')
best_epoch = 0

if do_training:
    print('Started Training')

    for epoch in range(15):  # number of epochs
        random.shuffle(train) # shuffle data every epoch
        train_loss, train_accuracy = train_model(net, create_batches(train, 64), optimizer, criterion, device)
        val_loss, val_accuracy = evaluate_model(net, create_batches(test, 64), criterion, device)

        train_losses.append(train_loss)
        train_accuracies.append(train_accuracy)
        val_losses.append(val_loss)
        val_accuracies.append(val_accuracy)

        if val_loss < best_loss:
          best_loss = val_loss
          best_epoch = epoch
          torch.save(net.state_dict(), SAVE_PATH) # Save the model


        print("Epoch: " + str(epoch) + ", Train Loss: " + str(train_loss) + ", Train Accuracy: " + str(train_accuracy) + \
            ", Val Loss: " + str(val_loss) + ", Val Accuracy: " + str(val_accuracy))


    print("Best Val Loss: " + str(best_loss) + " at Epoch: " + str(best_epoch))
                
    print('Finished Training')

Started Training
Epoch: 0, Train Loss: 3.447267251571423, Train Accuracy: 0.19780401681280377, Val Loss: 3.430316033989492, Val Accuracy: 0.37904942280264814
Epoch: 1, Train Loss: 3.4077004091388683, Train Accuracy: 0.41236757891734843, Val Loss: 3.3805393474270597, Val Accuracy: 0.46916937229759764
Epoch: 2, Train Loss: 3.3536054240870596, Train Accuracy: 0.49575493828899364, Val Loss: 3.32689153064381, Val Accuracy: 0.5159406565656566
Epoch: 3, Train Loss: 3.295292195934935, Train Accuracy: 0.5752938093267722, Val Loss: 3.2733273891487507, Val Accuracy: 0.5869588745061798
Epoch: 4, Train Loss: 3.2416426009938197, Train Accuracy: 0.6204687155745356, Val Loss: 3.2275097225651597, Val Accuracy: 0.6234442641337713
Epoch: 5, Train Loss: 3.196251704003, Train Accuracy: 0.6502203570707196, Val Loss: 3.1809383474215114, Val Accuracy: 0.6543650795715024
Epoch: 6, Train Loss: 3.1590693087747255, Train Accuracy: 0.6701507669717527, Val Loss: 3.1551020025002834, Val Accuracy: 0.6571744228854324


In [17]:
# Add out of set to test set
test = []
for i,lang in enumerate(in_set + out_of_set, 0):
    print(lang, "(In-set)" if lang in in_set else "(Out-of-set)")
    filepath = root_dir + lang + '/data/raw_mfcc_pitch_' + lang + '.25.ark'

    chunks = []
    for key, numpy_array in kaldiio.load_ark(filepath):
        chunks +=  np.split(numpy_array, np.arange(50000, len(numpy_array), 50000))
    random.shuffle(chunks)

    for j in range(len(chunks)):
        chunk = chunks[j]
        inputs = torch.from_numpy(np.expand_dims(chunk, axis=0))
        inputs.to(device)
        labels = torch.from_numpy(np.array([i if lang in in_set else -1]))
        labels.to(device)

        test.append((inputs,labels))
        
print(len(test))

ENG (In-set)
GER (In-set)
ICE (In-set)
FRE (In-set)
SPA (In-set)
ARA (In-set)
RUS (In-set)
BEN (In-set)
KAS (In-set)
GRE (In-set)
CAT (In-set)
KOR (In-set)
TUR (In-set)
TAM (In-set)
TEL (In-set)
CHI (In-set)
TIB (In-set)
JAV (In-set)
EWE (In-set)
HAU (In-set)
LIN (In-set)
YOR (In-set)
HUN (In-set)
HAW (In-set)
MAO (In-set)
ITA (In-set)
URD (In-set)
SWE (In-set)
PUS (In-set)
GEO (In-set)
HIN (In-set)
THA (In-set)
DUT (Out-of-set)
HEB (Out-of-set)
UKR (Out-of-set)
BUL (Out-of-set)
PER (Out-of-set)
ALB (Out-of-set)
UIG (Out-of-set)
MAL (Out-of-set)
BUR (Out-of-set)
IBA (Out-of-set)
ASA (Out-of-set)
AKU (Out-of-set)
ARM (Out-of-set)
HRV (Out-of-set)
FIN (Out-of-set)
JPN (Out-of-set)
NOR (Out-of-set)
NEP (Out-of-set)
RUM (Out-of-set)
6133


In [18]:
net.load_state_dict(torch.load(LOAD_PATH))

for thresh in [0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8]:
    correct = 0
    for x, y in test:
      # Send to GPU if available
      if device is not None:
        x = x.to(device)
        y = y.to(device)

      y_pred = net(x)
      y_pred = torch.mean(y_pred, 1)
      conf = y_pred.amax(1, keepdim=True)

      if conf > thresh:
          top_pred = y_pred.argmax(1, keepdim=True)
      else:
          top_pred = -1

      if top_pred == y:
        correct += 1
        
    print(thresh, correct / len(test))

0.2 0.4265449209196152
0.3 0.45361160932659383
0.4 0.48622207728680905
0.5 0.5144301320723952
0.6 0.4920919615196478
0.7 0.4053481167454753
0.8 0.28697211804989403


In [1]:
import kaldiio
import sys, os
import random
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import torch.optim as optim
from sklearn.metrics import confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
from pickle import Unpickler
from tdnn import TDNN
from collections import defaultdict

random.seed(1)



device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(device)

in_set = ['ENG', 'GER', 'ICE', 'FRE', 'SPA', 'ARA', 'RUS', 'BEN', 'KAS', 'GRE', 'CAT', 'KOR', 'TUR', 'TAM', 'TEL', 'CHI', 'TIB', 'JAV', 'EWE', 'HAU', 'LIN', 'YOR', 'HUN', 'HAW', 'MAO', 'ITA', 'URD', 'SWE', 'PUS', 'GEO', 'HIN', 'THA']
out_of_set = ['DUT', 'HEB', 'UKR', 'BUL', 'PER', 'ALB', 'UIG', 'MAL', 'BUR', 'IBA', 'ASA', 'AKU', 'ARM', 'HRV', 'FIN', 'JPN', 'NOR', 'NEP', 'RUM']

root_dir = "db/cu-multilang-dataset/"


data = []
for i,lang in enumerate(in_set + out_of_set, 0):
    print(lang, "(In-set)" if lang in in_set else "(Out-of-set)")
    
    for f_idx in range(1, 30 + 1):
        
        filepath = root_dir + lang + '/data/raw_mfcc_pitch_' + lang + '.' + str(f_idx) + '.ark'

        for key, numpy_array in kaldiio.load_ark(filepath):
            inputs = torch.from_numpy(np.expand_dims(numpy_array, axis=0))
            inputs.to(device)
            labels = torch.from_numpy(np.array([i if lang in in_set else -1]))
            labels.to(device)
            data.append((inputs, labels))


print("\ncombining all mfccs by label")
random.shuffle(data)
data_concatenated = dict()
for iter,i in enumerate(data): 
    label = i[1].numpy()[0]
    mfcc = np.squeeze(i[0].numpy(), axis=0)
    if label in data_concatenated:
        data_concatenated[label].append(mfcc)
    else:
        data_concatenated[label] = [mfcc]

for i in data_concatenated:
    data_concatenated[i] = np.vstack(data_concatenated[i])

del data

def chunkify_tensor(tensor, size=400):
    return torch.split(tensor, size, dim=1)[:-1] # except last one bc that isn't the right size

'''
Train (tdnn):       95% of in-set
Train (lda/plda):   80% of out-of-set
Test:               5% of in-set + 20% out-of-set
'''

print("chunking and splitting into 3 groups:")
train1, train2, test = [], [], []
for i in data_concatenated:
    label = torch.from_numpy(np.array([i]))
    mfcc = torch.from_numpy(np.expand_dims(data_concatenated[i], axis=0))
    chunks = chunkify_tensor(mfcc)
    if i >= 0:
        cutoff = int(len(chunks) * 0.95)
        for chunk in chunks[:cutoff]:
            train1.append((chunk.to(device), label.to(device)))
        for chunk in chunks[cutoff:]:
            test.append((chunk.to(device), label.to(device)))
    else:
        cutoff = int(len(chunks) * 0.8)
        for chunk in chunks[:cutoff]:
            train2.append((chunk.to(device), label.to(device)))
        for chunk in chunks[cutoff:]:
            test.append((chunk.to(device), label.to(device)))

del data_concatenated

print("\ttrain1:", len(train1))
print("\ttrain2:", len(train2))
print("\ttest:", len(test))
print("\t\ttest (in-set):", len([i for i in test if i[1].to('cpu').numpy()[0] >= 0]))
print("\t\ttest (oos):", len([i for i in test if i[1].to('cpu').numpy()[0] < 0]))
print()
print("labels in test:", set([i[1].to('cpu').numpy()[0] for i in test]))
print(test[0][0].size())
asdfasdfasdf = defaultdict(int)
for i in test:
    asdfasdfasdf[i[1].to('cpu').numpy()[0]] += 1
print(sorted(asdfasdfasdf.items()))
print()


print(test[:5])


class Net(nn.Module):
    def __init__(self, in_size, num_classes):
        super().__init__()
        
        self.layer1 = TDNN(input_dim=in_size, output_dim=256, context_size=3)
        self.layer2 = TDNN(input_dim=256, output_dim=256, context_size=3, dilation=1)
        self.layer3 = TDNN(input_dim=256, output_dim=256, context_size=3, dilation=1)
        self.layer4 = TDNN(input_dim=256, output_dim=256, context_size=1)
        self.layer5 = TDNN(input_dim=256, output_dim=256, context_size=1)
        self.final_layer = TDNN(input_dim=256, output_dim=num_classes, context_size=1)
        
    def forward(self, x):
        forward_pass = nn.Sequential(
            self.layer1,
            nn.ReLU(),
            self.layer2,
            nn.ReLU(),
            self.layer3,
            nn.ReLU(),
            self.layer4,
            nn.ReLU(),
            self.layer5,
            nn.ReLU(),
            self.final_layer)
        
        return forward_pass(x)



# LOAD_PATH = 'saved-models/tdnn-256-4s-25epochs'
print('Loading the model')

if len(sys.argv) != 2:
    print("Usage: python3 decode.py <model_name> \n Please refer to the saved_models directory to see available models and include the name without any extension.")
    exit(1)

model_name = sys.argv[1]
    
infile = open("saved_models/" + model_name + ".pickle", "rb")
net = Unpickler(infile).load()
infile.close()

print('Finished loading')
print()



#only test on the in-set ones
test = [i for i in test if i[1].to('cpu').numpy()[0] >= 0]
random.shuffle(test)
print(len(test), '\n')



print("Testing...")
correct, total = 0, 0

for i, data in enumerate(test,0):
    if i%2000 == 0 and total > 0:
        print("iter:", i, "\tacc:", correct/total)

    inputs, labels = data[0].to(device), data[1].to(device)
    
    outputs = net(inputs)
    outputs = torch.mean(outputs, 1)
    outputs = F.softmax(outputs, dim=1)
    predicted = outputs.argmax(1, keepdim=True)

    total += 1
    if predicted == labels:
        correct += 1

print("\nAcc:", correct/total)

cuda:0
ENG (In-set)
GER (In-set)
ICE (In-set)
FRE (In-set)
SPA (In-set)
ARA (In-set)
RUS (In-set)
BEN (In-set)
KAS (In-set)
GRE (In-set)
CAT (In-set)
KOR (In-set)
TUR (In-set)
TAM (In-set)
TEL (In-set)
CHI (In-set)
TIB (In-set)
JAV (In-set)
EWE (In-set)
HAU (In-set)
LIN (In-set)
YOR (In-set)
HUN (In-set)
HAW (In-set)
MAO (In-set)
ITA (In-set)
URD (In-set)
SWE (In-set)
PUS (In-set)
GEO (In-set)
HIN (In-set)
THA (In-set)
DUT (Out-of-set)
HEB (Out-of-set)
UKR (Out-of-set)
BUL (Out-of-set)
PER (Out-of-set)
ALB (Out-of-set)
UIG (Out-of-set)
MAL (Out-of-set)
BUR (Out-of-set)
IBA (Out-of-set)
ASA (Out-of-set)
AKU (Out-of-set)
ARM (Out-of-set)
HRV (Out-of-set)
FIN (Out-of-set)
JPN (Out-of-set)
NOR (Out-of-set)
NEP (Out-of-set)
RUM (Out-of-set)

combining all mfccs by label
chunking and splitting into 3 groups:
	train1: 246728
	train2: 86336
	test: 34586
		test (in-set): 13001
		test (oos): 21585

labels in test: {0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22,